In [335]:
import os
import pandas as pd
import numpy as np
import matplotlib

In [336]:
# Load all files into single dataframe
df = pd.DataFrame()
for file in os.listdir("energy-analyst-data-exercise-public/historicalPriceData"):
    with open(os.path.join("energy-analyst-data-exercise-public/historicalPriceData/", file)) as f:
        df1 = pd.read_csv(f, skiprows=1, header=None, names=['Date','SettlementPoint','Price'])
        df = df.append(df1)

In [337]:
df.head()

,Date,SettlementPoint,Price
0,2016-01-01 00:00:00,HB_BUSAVG,18.42
1,2016-01-01 00:00:00,HB_HOUSTON,18.42
2,2016-01-01 00:00:00,HB_HUBAVG,18.42
3,2016-01-01 00:00:00,HB_NORTH,18.43
4,2016-01-01 00:00:00,HB_SOUTH,18.38


In [338]:
# Task 2: Compute the average price for each settlement point and year-month
df['Date'] = pd.to_datetime(df['Date']) # Convert string to datetime
df['Month'] = pd.DatetimeIndex(df['Date']).month # Extract month from datetime
df['Year'] = pd.DatetimeIndex(df['Date']).year # Extract year from datetime
averages = df.groupby(['SettlementPoint', 'Year', 'Month'])['Price'].mean() # Calculate average price by settlement point, year, and month

In [339]:
# Task 3: Write computed averages to CSV file
averages.to_csv('output/AveragePriceByMonth.csv')

In [340]:
# Task 4: Compute volatility of prices for each settlement point and year
# Volatility is standard deviation of natural log of prices
# Make new dataframe with no negative prices and no LZ_ prefix
df_volatility = df[(df['Price'] > 0) & (df['SettlementPoint'].str.contains('LZ_') == False)]
# Standard deviation of log of hourly price
df_volatility['LogPrice'] = np.log(df_volatility['Price'])
df_volatility = df_volatility.groupby(['SettlementPoint', 'Year'])['LogPrice'].std()
# TODO: Fix rename statement (not working)

/tmp/ipykernel_221477/1013591396.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_volatility['LogPrice'] = np.log(df_volatility['Price'])


In [341]:
# Task 5: Write computed volatility to CSV file
df_volatility.to_csv('output/HourlyPrice1VolatilityByYear.csv')

In [342]:
# Get highest volatility for each settlement point and year
df_volatility_max = df_volatility.groupby(['Year']).max()
# TODO: Get settlement name as well

In [343]:
df_volatility_max.to_csv('output/MaxVolatilityByYear.csv')

In [347]:
# Task 7: Converting to necessary CQuant format
# Col1: SettlementPoint
# Col2: Date
# Cols3-26: Hourly prices
# Loop every SettlementPoint
for SettlementPoint in df.SettlementPoint.unique():
    df_temp = df[df['SettlementPoint'] == SettlementPoint] # Get data from settlement point
    df_temp['Hour'] = pd.DatetimeIndex(df_temp['Date']).hour # Get hour from datetime
    df_temp['Day'] = pd.DatetimeIndex(df_temp['Date']).day # Get hour from datetime
    df_temp['Date2'] = pd.to_datetime(df_temp['Date']).dt.date # Get Date
    df_temp_2 = pd.DataFrame() # Create new dataframe for the SettlementPoint
    # Create Columns
    df_temp_2.insert(0, 'Variable', '')
    df_temp_2.insert(1, 'Date', '')
    for i in range(1,25):
        df_temp_2.insert(i+1, 'X'+str(i), '')
    # Get all entries for one date
    for j in df_temp['Date2'].unique():
        day_prices = [SettlementPoint, j]
        for i in range(0, 24):
             day_prices.append(df_temp[df_temp['Date'] == pd.to_datetime(str(j) + " " + str(i) + ":00:00")]['Price'].values[0])
        df_temp_2.loc[len(df_temp_2)] = day_prices
        print(df_temp_2)
    # Write to CSV
    df_temp_2.to_csv('output/formattedSpotHistory/' + SettlementPoint + '.csv', index=False)

/tmp/ipykernel_221477/2475889985.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Hour'] = pd.DatetimeIndex(df_temp['Date']).hour # Get hour from datetime
/tmp/ipykernel_221477/2475889985.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['Day'] = pd.DatetimeIndex(df_temp['Date']).day # Get hour from datetime
/tmp/ipykernel_221477/2475889985.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

    Variable        Date     X1     X2     X3     X4     X5     X6     X7  \
0  HB_BUSAVG  2016-01-01  18.42  16.15  15.61  15.62  15.94  16.69  18.96   

      X8  ...    X15    X16    X17    X18    X19    X20    X21    X22    X23  \
0  19.18  ...  19.09  19.34  20.07  24.96  26.26  23.24  22.79  21.36  20.31   

     X24  
0  19.65  

[1 rows x 26 columns]
    Variable        Date     X1     X2     X3     X4     X5     X6     X7  \
0  HB_BUSAVG  2016-01-01  18.42  16.15  15.61  15.62  15.94  16.69  18.96   
1  HB_BUSAVG  2016-01-02  18.99  17.94  17.16  17.00  17.90  17.65  20.11   

      X8  ...    X15    X16    X17    X18    X19    X20    X21    X22    X23  \
0  19.18  ...  19.09  19.34  20.07  24.96  26.26  23.24  22.79  21.36  20.31   
1  21.33  ...  18.29  17.94  19.34  23.48  23.48  21.73  20.73  19.97  19.87   

     X24  
0  19.65  
1  18.43  

[2 rows x 26 columns]
    Variable        Date     X1     X2     X3     X4     X5     X6     X7  \
0  HB_BUSAVG  2016-01-01  18.42  

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Bonus Task 1: Mean Plots
df_avg = pd.read_csv('output/AveragePriceByMonth.csv')
df_avg_HB = df_avg[df_avg['SettlementPoint'].str.contains('HB_') == True].groupby(['Year', 'Month']).mean()
print(df_avg_HB.head())
# Plot df_avg_HB as line plot and df_avg as scatter plot

ax = df_avg_HB.plot(kind='line', title='Average Price by Month')
df_avg_LZ = df_avg[df_avg['SettlementPoint'].str.contains('LZ_') == True].groupby(['Year', 'Month']).mean()